In [1]:
import pandas as pd
import pdb
import numpy as np
import itertools
from utils import *
from sm_utils import *

from snakemake.io import expand
import yaml

In [2]:
config_file = 'config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

## 240129

In [3]:
configfile: 'config.yml'
config_tsv = 'config.tsv'
p_meta_tsv = 'pseudochromosome_metadata.tsv'
meta_tsv = 'mouse_metadata.tsv'
an_meta_tsv = 'analysis_config.tsv'
auto_dedupe = True

df, p_df = parse_config_file_analysis(config_tsv,
                       meta_tsv,
                       p_meta_tsv,
                       an_meta_tsv,
                       auto_dedupe=auto_dedupe)

In [4]:
len(p_df.index)

91

In [ ]:
expand(config['ref']['cerberus']['ends'],
               end_mode=end_modes)

In [6]:
df.cerberus_run.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
       '13', '14', '15', '16', '17', '18', '19'], dtype=object)